
# Lalal.ai Downloader and Merger

This notebook provides a Python-based solution for downloading audio segments exported from Lalal.ai and merging them into a single audio file. It supports both vocals and no-vocals playlists and works seamlessly in Google Colab.

## Features
- **URL or File ID Input**: Accepts either a full URL or a file ID for processing.
- **Flexible Playlist Selection**: Allows users to choose between vocals or no-vocals playlists.
- **Automated Download**: Sequentially downloads all available segments.
- **Audio Merging**: Combines downloaded segments into a single MP3 file.
- **Colab Integration**: Designed for use in Google Colab with built-in file download support.

## Prerequisites
Before running the notebook, install the necessary dependencies:
```bash
!pip install pydub
```

## How to Use
1. **Extract File ID**:
   - Open the developer console in your browser (e.g., press `F12` in Chrome).
   - Go to the **Network** tab and play the converted song.
   - Locate the segment URLs and right click, copy and copy the URL.

2. **Run the Notebook**:
   - Input either a full URL or the extracted file ID when prompted.

3. **Choose Playlist**:
   - Specify whether you want the vocals (`yes`) or no-vocals (`no`) playlist.

4. **Download and Merge**:
   - The script will download all segments and merge them into `merged_audio.mp3`.
   - The merged file will be available for download directly from Colab.

## Example Input
- URL:  
  `https://d.lalal.ai/media/split/298a10ab-c995-488b-899c-3c81e2118c9c/ebb93a82-85ba-4f9f-b8d1-2b675e021bef/vocals_playlist/segment-000.mp3`
- File ID:  
  `298a10ab-c995-488b-899c-3c81e2118c9c/ebb93a82-85ba-4f9f-b8d1-2b675e021bef`

## Output
- Individual segments downloaded to the `segments` directory.
- Merged audio saved as `merged_audio.mp3`.

## Notes
- Ensure the song is fully converted on Lalal.ai to access all segments. The script won't work on preview segments.
- The script automatically downloads and merges available segments.


# Lalal.ai Downloader
This is a simple python script that will download the lalal exported segments and merge them for you.

To get to the id of the song you go to the *Developers console* in your browser (F12 in chrome) and in network tab and play the song, you will see segments downloaded. **You have to have a full song converted or you will only get a preview**. Then click on the segment,

In [2]:
!pip install pydub

In [9]:
import requests
from pydub import AudioSegment
import os
from google.colab import files
import re

# Directory to store downloaded segments
DOWNLOAD_DIR = "segments"
MERGED_FILE = "merged_audio.mp3"

# Ensure the download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def extract_file_id(input_str):
    """Extract the file ID from the provided URL or return it if it's already an ID."""
    if re.match(r"^[a-f0-9\-/]{36,}$", input_str):
        return input_str  # Input is already an ID
    match = re.search(r"/media/split/(.*?/.*?)/", input_str)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid input. Provide a valid URL or file ID.")

def get_base_url(file_id, vocals):
    """Construct the base URL based on file ID and vocals/no vocals."""
    vocals_path = "no_vocals_playlist" if not vocals else "vocals_playlist"
    return f"https://d.lalal.ai/media/split/{file_id}/{vocals_path}/segment-"

def download_segment(base_url, segment_number):
    """Download a single segment by its number."""
    url = f"{base_url}{segment_number:03}.mp3"
    file_path = os.path.join(DOWNLOAD_DIR, f"segment-{segment_number:03}.mp3")

    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(file_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):
                    file.write(chunk)
            print(f"Downloaded: {file_path}")
            return file_path
        else:
            print(f"Segment {segment_number} not found.")
            return None
    except Exception as e:
        print(f"Error downloading segment {segment_number}: {e}")
        return None

def merge_segments(segment_files):
    """Merge a list of audio segments into a single file."""
    merged = AudioSegment.empty()

    for file in segment_files:
        audio = AudioSegment.from_file(file)
        merged += audio

    merged.export(MERGED_FILE, format="mp3")
    print(f"Merged audio saved as {MERGED_FILE}")
    files.download(MERGED_FILE)

def main():
    input_str = input("Enter the URL of the segment or ID: ").strip()
    try:
        file_id = extract_file_id(input_str)
        print(f"File id found: {file_id}")
    except ValueError as e:
        print(e)
        return




    vocals_input = input("Output vocals? (yes/no): ").strip().lower()
    vocals = vocals_input == "yes" or vocals_input == "y"

    base_url = get_base_url(file_id, vocals)

    segment_files = []
    segment_number = 0

    while True:
        file_path = download_segment(base_url, segment_number)
        if file_path:
            segment_files.append(file_path)
        else:
            break
        segment_number += 1

    if segment_files:
        merge_segments(segment_files)
    else:
        print("No segments downloaded.")

if __name__ == "__main__":
    main()


Enter the URL of the segment or ID: 298a10ab-c995-488b-899c-3c81e2118c9c/ebb93a82-85ba-4f9f-b8d1-2b675e021bef
298a10ab-c995-488b-899c-3c81e2118c9c/ebb93a82-85ba-4f9f-b8d1-2b675e021bef
